In [1]:
%matplotlib inline
import numpy as np
import scipy.integrate 
import scipy.interpolate
import matplotlib.pyplot as plt
import constrain_IA_Blazek_FudgeFactor as Bl

SURVEY = 'LSST_DESI'

if SURVEY=='SDSS':
    import params as pa
elif SURVEY=='LSST_DESI':
    import params_LSST_DESI as pa
else:
    print "We don't have support for that survey yet"

In [2]:
# Get boost factors
Boost_a = Bl.get_boost(Bl.rp_cent, pa.boost_close)
Boost_b = Bl.get_boost(Bl.rp_cent, pa.boost_far)

print "Boost_a=", Boost_a
print "Boost_b=",Boost_b

Boost_a= [ 1.23402169  1.11799886  1.05949761  1.03        1.01512666  1.00762719
  1.0038458 ]
Boost_b= [ 1.00000078  1.00000039  1.0000002   1.0000001   1.00000005  1.00000003
  1.00000001]


In [3]:
# gamma_IA_fiducial, from model
g_IA_fid = Bl.gamma_fid(Bl.rp_cent)
print "g_IA_fid=", g_IA_fid

Loading wgg 1halo term from file.
Loading wgg 2halo term from file.
Loading wgp 1halo term from file
Loading wgp 2halo term from file 
g_IA_fid= [  4.51272489e-05   6.78984781e-05   1.16010714e-04   1.89835065e-04
   8.52851822e-05   4.55268495e-05   3.05672122e-05]


In [4]:
# Load other quantities
(rp_bins_c, F_a_fid, F_b_fid) = np.loadtxt('./txtfiles/F_afid_bfid_survey='+pa.survey+'.txt', unpack=True)
(rp_bins_c, Sig_IA_a_fid, Sig_IA_b_fid) = np.loadtxt('./txtfiles/Sig_IA_afid_bfid_survey='+pa.survey+'.txt', unpack=True)
(cz_a_fid, cz_b_fid) = np.loadtxt('./txtfiles/cz_afid_bfid_survey='+pa.survey+'.txt', unpack=True)

In [5]:
# Get Delta Sigma Ests
DeltaSig_est_a = Bl.get_est_DeltaSig(pa.zeff, Bl.rp_bins, Bl.rp_cent, Boost_a, F_a_fid, cz_a_fid, Sig_IA_a_fid, g_IA_fid)
DeltaSig_est_b = Bl.get_est_DeltaSig(pa.zeff, Bl.rp_bins, Bl.rp_cent, Boost_b, F_b_fid, cz_b_fid, Sig_IA_b_fid, g_IA_fid)

print "Delta Sig=", DeltaSig_est_a
print "Delta Sig b=", DeltaSig_est_b

Delta Sig= [ 102.59998743   76.41640813   45.4643212    21.69303438    8.33492492
    2.99542039    1.35911403]
Delta Sig b= [ 103.02593211   76.72493757   45.60785488   21.70636232    8.3478216
    2.99833904    1.35886423]


In [6]:
# These are the shape-noise-dominated diagonal covariance matrices associated with each sample. Units: Msol^2 h / pc^2, comoving.
DeltaCov_a = Bl.shapenoise_cov(pa.e_rms_Bl_a, pa.zeff, pa.zeff+pa.delta_z, Boost_a, Bl.rp_cent, Bl.rp_bins, pa.dNdzpar_fid, pa.pzpar_fid,  pa.dNdztype, pa.pztype)
DeltaCov_b = Bl.shapenoise_cov(pa.e_rms_Bl_b, pa.zeff+pa.delta_z, pa.zphmax, Boost_b, Bl.rp_cent, Bl.rp_bins, pa.dNdzpar_fid, pa.pzpar_fid,  pa.dNdztype, pa.pztype)

print "DeltaCov_a=", DeltaCov_a
print "DeltaCov_b=", DeltaCov_b

DeltaCov_a= [  2.77870710e+01   5.53705057e+00   1.05480980e+00   1.95880500e-01
   3.58808303e-02   6.52585058e-03   1.18256300e-03]
DeltaCov_b= [  3.29161853e+01   5.94242835e+00   1.07279892e+00   1.93674596e-01
   3.49644715e-02   6.31220750e-03   1.13955571e-03]


In [7]:
# Get the statistical error 
boost_err_a = 0.1 * (Boost_a-1.); boost_err_b = 0.1 * (Boost_b-1.);

gammaIA_stat_cov_tot = np.zeros((len(Bl.rp_cent), len(Bl.rp_cent)))
gammaIA_stat_cov_boost = np.zeros((len(Bl.rp_cent), len(Bl.rp_cent)))
for i in range(0,len((Bl.rp_cent))):
    num_term_stat_tot = ( cz_a_fid**2 * DeltaCov_a[i]  + cz_b_fid**2 * DeltaCov_b[i] ) / (( cz_a_fid * DeltaSig_est_a[i]) - ( cz_b_fid * DeltaSig_est_b[i]))**2
    denom_term_stat_tot = ( ( cz_a_fid * (Boost_a[i] -1. + F_a_fid[i]) * Sig_IA_a_fid[i])**2 * (boost_err_a[i]**2  / (Boost_a[i] -1. + F_a_fid[i])**2 ) + ( cz_b_fid * (Boost_b[i] -1. + F_b_fid[i]) * Sig_IA_b_fid[i])**2 * ( boost_err_b[i]**2   / (Boost_b[i] -1. + F_b_fid[i])**2 ) ) / ( ( cz_a_fid * (Boost_a[i] -1. + F_a_fid[i]) * Sig_IA_a_fid[i]) -  ( cz_b_fid * (Boost_b[i] -1. + F_b_fid[i]) * Sig_IA_b_fid[i]) )**2
    num_term_stat_Boost = 0.
    denom_term_stat_Boost = ( ( cz_a_fid * (Boost_a[i] -1. + F_a_fid[i]) * Sig_IA_a_fid[i])**2 * (boost_err_a[i]**2  / (Boost_a[i] -1. + F_a_fid[i])**2 ) + ( cz_b_fid * (Boost_b[i] -1. + F_b_fid[i]) * Sig_IA_b_fid[i])**2 * ( boost_err_b[i]**2   / (Boost_b[i] -1. + F_b_fid[i])**2 ) ) / ( ( cz_a_fid * (Boost_a[i] -1. + F_a_fid[i]) * Sig_IA_a_fid[i]) -  ( cz_b_fid * (Boost_b[i] -1. + F_b_fid[i]) * Sig_IA_b_fid[i]) )**2
    gammaIA_stat_cov_tot[i,i] = g_IA_fid[i]**2 * (num_term_stat_tot + denom_term_stat_tot)
    gammaIA_stat_cov_boost[i,i] = g_IA_fid[i]**2 * (num_term_stat_Boost + denom_term_stat_Boost)
    print "num_term_stat_tot=", num_term_stat_tot
    
print "tot=", np.diag(gammaIA_stat_cov_tot)
print "boost=",np.diag(gammaIA_stat_cov_boost) 

num_term_stat_tot= 3583.12452567
num_term_stat_tot= 1029.01926434
num_term_stat_tot= 201.505010418
num_term_stat_tot= 35.9922103248
num_term_stat_tot= 68.2549019123
num_term_stat_tot= 72.6896905933
num_term_stat_tot= 40.2068193121
tot= [  7.29693803e-06   4.74402241e-06   2.71203030e-06   1.29720091e-06
   4.96470860e-07   1.50665232e-07   3.75676938e-08]
boost= [  1.74677476e-11   3.43707632e-11   7.79889132e-11   1.36967879e-10
   1.45779182e-11   1.74467810e-12   2.72787465e-13]


In [8]:
# The fraction of the total statistical error which is due to boost statistical
frac = np.diag(gammaIA_stat_cov_boost) / np.diag(gammaIA_stat_cov_tot)
print "percentage=", frac*100

percentage= [ 0.00023938  0.00072451  0.00287567  0.01055873  0.00293631  0.00115798
  0.00072612]
